In [1]:
import math
import datetime
import numpy as np
import pickle

import methodsMLinterns
import customAutoencoder

Using TensorFlow backend.


In [2]:
stocks = ['DNB', 'NRG', 'CL', 'ANTM', 'NEE', 'PAYX', 'VAR', 'NI', 'MNST', 'JNJ', 'TGNA', 'NOV', 'FIS', 'BLK', 'HBI', 'NVDA', 'DLTR', 'MRO', 'EMN', 'AMT', 'FLR', 'IBM', 'BK', 'NFX', 'AGN', 'LRCX', 'DIS', 'LH', 'C', 'MNK']

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [4]:
features_1p4_extra =['aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'abj', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acr', 'acw', 'acx', 'acy', 'adi', 'adj', 'adl', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aex', 'aey', 'aez', 'afa', 'afj', 'afl', 'afo', 'afp', 'afq', 'afr', 'afs', 'aft', 'afu', 'afv', 'afw', 'afx', 'afy', 'afz', 'aga', 'agb', 'agc', 'agd', 'age', 'agf', 'agg', 'agh', 'agi', 'agj', 'agk', 'agl', 'agm', 'agn', 'ago', 'agp', 'agq', 'agr', 'ags', 'agt', 'agu', 'agv', 'agw', 'agx', 'agy', 'ahf', 'ahg', 'ahh', 'ahi', 'ahj', 'ahk', 'ahl', 'ahm', 'ahn', 'aho', 'zhq', 'zhr', 'zhs', 'zht', 'zhu', 'zhv', 'zhw', 'ziy', 'zjb', 'zjc', 'zjd', 'zje', 'zjf', 'zjg', 'zjh', 'zji', 'zjj', 'zjk', 'zjl', 'zjm', 'zjn', 'zjo', 'zjp', 'zjq', 'zjr', 'zjs', 'zjt', 'zju', 'zkg', 'zkl', 'zkm', 'zkn', 'zkx', 'zky', 'zla', 'zld', 'zle', 'zlf', 'zlg', 'zlh', 'zli', 'zlj', 'zlk', 'zll', 'zlm', 'zln', 'zlo', 'zlp', 'zlq', 'zlr', 'zls', 'zlt', 'zlu', 'zlv', 'zlw', 'zlx', 'zly', 'zlz', 'zma', 'zmb', 'zmc', 'zmd', 'zme', 'zmf', 'zmg', 'zmh', 'zmm', 'zmn', 'zmo', 'zmp', 'zmy', 'zna', 'znd', 'zne', 'znf', 'zng', 'znh', 'zni', 'znj', 'znk', 'znl', 'znm', 'znn', 'zno', 'znp', 'znq', 'znr', 'zns', 'znt', 'znu', 'znv', 'znw', 'znx', 'zny', 'znz', 'zoa', 'zob', 'zoc', 'zod', 'zoe', 'zof', 'zog', 'zoh', 'zoi', 'zoj', 'zok', 'zol', 'zom', 'zon', 'zou', 'zov', 'zow', 'zox', 'zoy', 'zoz', 'zpa', 'zpb', 'zpc', 'zpd', 'zpe']
features_1p4_extra_a=[f for f in features_1p4_extra if f[0]=='a']
features_1p4_extra_z=[f for f in features_1p4_extra if f[0]=='z']
assert(len(features_1p4_extra_a)+len(features_1p4_extra_z)==len(features_1p4_extra))

In [5]:
random_state = 0
Cs = np.logspace(-4, 5)
cv = 5
ratio_threshold = 0.65
date_test_set = datetime.date(2016, 1, 1)

In [6]:
clf_portfolio_dic = methodsMLinterns.ClassificationPortfolio(stocks=stocks, minutes_forward=30)
clf_portfolio_dic.loadData()
clf_portfolio_dic.cleanUpData(features_1p4_extra)
clf_portfolio_dic.getTrainTestSetDate(date_test_set)

### Group together all the stocks

In [7]:
def prepareData(features):
    X_train = np.array([], dtype=np.float64).reshape(0,len(features))
    y_train = np.array([], dtype=np.float64).reshape(0,1)
    X_test = np.array([], dtype=np.float64).reshape(0,len(features))
    y_test = np.array([], dtype=np.float64).reshape(0,1)


    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = stock + str(clf_portfolio_dic.minutes_forward)
        if k==0:
            X_train, y_train = clf_portfolio_dic.X_train_dic[name][features].as_matrix(),(clf_portfolio_dic.y_train_dic[name]+1)/2
            X_test, y_test = clf_portfolio_dic.X_test_dic[name][features].as_matrix(), (clf_portfolio_dic.y_test_dic[name]+1)/2
        else:
            X_train = np.concatenate((X_train,clf_portfolio_dic.X_train_dic[name][features].as_matrix()),axis=0)
            y_train = np.concatenate((y_train,(clf_portfolio_dic.y_train_dic[name]+1)/2),axis=0)
            X_test = np.concatenate((X_test,clf_portfolio_dic.X_test_dic[name][features].as_matrix()),axis=0)
            y_test = np.concatenate((y_test,(clf_portfolio_dic.y_test_dic[name]+1)/2),axis=0)

    print(X_train.shape)
    # Transform to one hot vectors
    y_t = np.zeros((y_train.shape[0], 2))
    y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
    y_train = y_t

    y_t = np.zeros((y_test.shape[0], 2))
    y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
    y_test = y_t
    
    return X_train,y_train,X_test,y_test

### Use Keras

In [15]:
ds=[0,0.2,0.5,0.8]
ls=[0,0.01,0.1]
#ds=[0.5]
#ls=[0.01]

Iterate over the different models

In [18]:
# Define the NN architecture
features = features_1p4_extra_z
architecture = [len(features),100,70]
print(architecture)
X_train,y_train,X_test,y_test = prepareData(features)

for d in ds:
    for l in ls:
        print("\n\n\t (d%s,l%s) First part:\n"%(d,l))
        auto = customAutoencoder.Autoencoder(architecture,dropout=d,inputNoise=0.1,l1reg=l,l2reg=l)
        auto.buildAutoencoder()
        
        auto.fit(X_train,y_train,epochs=400)
        chkpt_0 = "/Users/clement/cqc/weights/%s.hdf5"%(auto.name)
        
        auto.freezeAutoencoder(True)
        
        print("\n\n\t (d%s,l%s) Second part:\n"%(d,l))
        auto.loadFromWeights(chkpt_0)
        auto.fit(X_train,y_train,epochs=100)
        

[120, 100, 70]
(122607, 120)


	 (d0.5,l0.01) First part:

Epoch 00000: val_decoded_loss improved from inf to -0.00376, saving model to /Users/clement/cqc/weights/A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_f0.hdf5
Epoch 00001: val_decoded_loss improved from -0.00376 to -0.00522, saving model to /Users/clement/cqc/weights/A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_f0.hdf5
Epoch 00002: val_decoded_loss improved from -0.00522 to -0.00567, saving model to /Users/clement/cqc/weights/A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_f0.hdf5
Epoch 00003: val_decoded_loss improved from -0.00567 to -0.00589, saving model to /Users/clement/cqc/weights/A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_f0.hdf5
Epoch 00004: val_decoded_loss improved from -0.00589 to -0.00605, saving model to /Users/clement/cqc/weights/A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_f0.hdf5
Epoch 00005: val_decoded_loss improved from -0.00605 to -0.00618, saving model to /Users/clement/cqc/weights/A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_

In [24]:
print("hello")

hello


### Test on stocks separately

In [20]:
def testOnSeparateStocks(auto,feat):
    acc_train_autoencoder = np.zeros(clf_portfolio_dic.N_stocks)
    acc_test_autoencoder = np.zeros(clf_portfolio_dic.N_stocks)
    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = "%s30"%stock
        print(k,name)
    
        x_train = np.array(clf_portfolio_dic.X_train_dic[name][feat])
        x_test = np.array(clf_portfolio_dic.X_test_dic[name][feat])
        y_train = (clf_portfolio_dic.y_train_dic[name]+1)/2
        y_test = (clf_portfolio_dic.y_test_dic[name]+1)/2

        y_t = np.zeros((y_train.shape[0], 2))
        y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
        y_train = y_t

        y_t = np.zeros((y_test.shape[0], 2))
        y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
        y_test = y_t
    
        y_pred_train = auto.predict(x_train)
        y_pred_test = auto.predict(x_test)
        acc_train = (100 * (np.argmax(y_pred_train,1) == np.argmax(y_train,1))).mean()
        acc_test = (100 * (np.argmax(y_pred_test,1) == np.argmax(y_test,1))).mean()
        acc_train_autoencoder[k] = acc_train
        acc_test_autoencoder[k] = acc_test
    return(acc_train_autoencoder,acc_test_autoencoder)

In [21]:
expList = []

for feat in [features_1p4_extra_z]:#[features_1p4_extra_z,features_1p4_extra,features_1p4_extra_a]:
    for d in ds:
        for l in ls:
            for i in [70]:
                for f in [0,1]:
                    name = "A30S_(%s,%s,%s)_d%s_in%s_r(%s,%s)_f%s"%(len(feat), 100, i, d, 0.1, l, l, f)
                    print(name)
                    e = methodsMLinterns.ExperimentPerformance(
                        methodName=name, 
                        stocks=stocks, originalFeatures=feat)
                    
                    auto = customAutoencoder.Autoencoder([len(feat),100,i],dropout=d,l1reg=l,l2reg=l)
                    auto.buildAutoencoder()
                    if f == 1:
                        auto.freezeAutoencoder(True)
                    chkpt_0 = "/Users/clement/cqc/weights/%s.hdf5"%(name)
                    auto.loadFromWeights(chkpt_0)
                    acc_train_vec,acc_test_vec = testOnSeparateStocks(auto,feat)
                    
                    e.setTrainResults(acc_train_vec)
                    e.setTestResults(acc_test_vec)
                    expList.append(e)

A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_f0
0 DNB30
1 NRG30
2 CL30
3 ANTM30
4 NEE30
5 PAYX30
6 VAR30
7 NI30
8 MNST30
9 JNJ30
10 TGNA30
11 NOV30
12 FIS30
13 BLK30
14 HBI30
15 NVDA30
16 DLTR30
17 MRO30
18 EMN30
19 AMT30
20 FLR30
21 IBM30
22 BK30
23 NFX30
24 AGN30
25 LRCX30
26 DIS30
27 LH30
28 C30
29 MNK30
A30S_(120,100,70)_d0.5_in0.1_r(0.01,0.01)_f1
0 DNB30
1 NRG30
2 CL30
3 ANTM30
4 NEE30
5 PAYX30
6 VAR30
7 NI30
8 MNST30
9 JNJ30
10 TGNA30
11 NOV30
12 FIS30
13 BLK30
14 HBI30
15 NVDA30
16 DLTR30
17 MRO30
18 EMN30
19 AMT30
20 FLR30
21 IBM30
22 BK30
23 NFX30
24 AGN30
25 LRCX30
26 DIS30
27 LH30
28 C30
29 MNK30


In [22]:
print(len(expList))

2


In [23]:
with open("pickles/acc_autoencoder_logit_70_z_denoising.p",'wb') as f:
    pickle.dump( expList, f, protocol=pickle.HIGHEST_PROTOCOL)